In [1]:
import requests
import csv
import json
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

In [2]:
site_id = '06719505'

hist_req = requests.get("https://waterservices.usgs.gov/nwis/dv/?format=json&sites={}&period=P520W&statCd=00003&siteStatus=all".format(site_id))
recent_req = requests.get("https://waterservices.usgs.gov/nwis/dv/?format=json&sites={}&startDT=2020-01-01&endDT=2020-05-13&statCd=00003&siteStatus=all".format(site_id))

In [3]:
# recent data
res = recent_req.json()

rec_days = []
rec_cfss = []

site_name = res["value"]["timeSeries"][0]["sourceInfo"]["siteName"]
lat = res["value"]["timeSeries"][0]["sourceInfo"]["geoLocation"]["geogLocation"]["latitude"]
lon = res["value"]["timeSeries"][0]["sourceInfo"]["geoLocation"]["geogLocation"]["longitude"]

for vals in res["value"]["timeSeries"][0]["values"][0]["value"]:
    cfs = round(float(vals["value"]),0)
    day = vals["dateTime"]
    date = datetime.strptime(day.split("T")[0], "%Y-%m-%d")
    doy = datetime.strftime(date, "%B %d")
    montInt = datetime.strftime(date, "%m")
    rec_days.append("{} {}".format(montInt, doy))
    rec_cfss.append(cfs)

# historical data
res = hist_req.json()
his_days = []
his_cfss = []
for vals in res["value"]["timeSeries"][0]["values"][0]["value"]:
    cfs = round(float(vals["value"]),0)
    day = vals["dateTime"]
    date = datetime.strptime(day.split("T")[0], "%Y-%m-%d")
    doy = datetime.strftime(date, "%B %d")
    montInt = datetime.strftime(date, "%m")
    his_days.append("{} {}".format(montInt, doy))
    his_cfss.append(cfs)

In [4]:
rec_d = {'day': rec_days, 'cfs': rec_cfss}
rec_df = pd.DataFrame(data=rec_d)

his_d = {'day': his_days, 'cfs': his_cfss}
his_df = pd.DataFrame(data=his_d)

In [23]:
stat = "mean"

rec_cfs_df = rec_df.groupby(['day']).agg({"cfs": ["{}".format(stat)]}).reset_index()

cols = rec_cfs_df.columns.tolist()

rec_cfs_df["mean_cfs"] = rec_cfs_df[cols[1]]

his_cfs_df = his_df.groupby(['day']).agg({"cfs": ["{}".format(stat)]}).reset_index()

cols = his_cfs_df.columns.tolist()

his_cfs_df["mean_cfs"] = his_cfs_df[cols[1]]

In [24]:
rec_cfs_df[['month', 'new_day']] = rec_cfs_df.day.str.split(expand=True, n=1)
his_cfs_df[['month', 'new_day']] = his_cfs_df.day.str.split(expand=True, n=1)

In [25]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=his_cfs_df['new_day'], y=his_cfs_df['mean_cfs'], name='{} CFS for the Day<br>(Over Last 10 Years)'.format(stat.title()),
                         line=dict(color='orange', width=2)))

fig.add_trace(go.Scatter(x=rec_cfs_df['new_day'], y=rec_cfs_df['mean_cfs'], name='{} CFS for the Day<br>(2020)'.format(stat.title()),
                         line=dict(color='blue', width=2)))

fig.update_layout(
    title_text="CFS for {}<br>Coordinates: {}, {}".format(site_name.title(), lat, lon),
    font=dict(family='Courier New, monospace', size=12, color='#7f7f7f'),
    yaxis_title="Cubic Feet per Second (CFS)"
)
fig.show()

In [27]:
fig.write_html(r"Z:\Dev\CFS_Charts\index.html")